In [1]:
import random

import pandas as pd
import torch
import wandb

from helper import *
from heterognn import *

The State of the nth node is expressed by 4 real scalars:

v_n -> the voltage at the node
delta_n -> the voltage angle at the node (relative to the slack bus)
p_n -> the net active power flowing into the node
q_n -> the net reactive power flowing into the node


The physical characteristics of the network are described by the power flow equations:

p = P(v, delta, W)
q = Q(v, delta, W)

-> Relate local net power generation with the global state
-> Depends on the topology W of the grid

Electrical grid => Weighted Graph

Nodes in the graph produce/consume power

Edges represent electrical connections between nodes

State Matrix X element of  R(N x 4) => graph signal with 4 features
    => Each row is the state of the corresponding Node

Adjacency Matrix A => sparse matrix to represent the connections of each node, element of R(N x N), Aij = 1 if node i is connected to node j


We use the GNN as a mode phi(X, A, H)

We want to imitate the OPF solution p*
-> We want to minimize a loss L over a dataset T = {{X, p*}}

Objective Function:min arg H of sum over T L(p*,phi(X, A, H)) and we use L = Mean Squared error

Once GNN model phi is trained, we do not need the costly p* from pandapower to make predictions

Input Data X - R^(Nx4): Uniformly sample p_ref and q_ref of each load L with P_L ~ Uniform(0.9 * p_ref, 1.1 * p_ref) and Q_L ~ Uniform(0.9 * q_ref, 1.1 * q_ref)

Pseudocode for X and y in supervised learning:
for each P_L and Q_L:
    Create X with sub-optimal DCOPF results
    Create y with Pandapower calculating p* ACOPF with IPOPT


Spatio-Temporal GNN -> superposition of a gnn with spatial info and a temporal layer (Temporal Conv,LSTM etc.) ??

Bus => Node in GNN








In [8]:
# get lists of simbench codes
all_simbench_codes = sb.collect_all_simbench_codes()
all_simbench_codes

['1-complete_data-mixed-all-0-sw',
 '1-complete_data-mixed-all-1-sw',
 '1-complete_data-mixed-all-2-sw',
 '1-EHVHVMVLV-mixed-all-0-sw',
 '1-EHVHVMVLV-mixed-all-1-sw',
 '1-EHVHVMVLV-mixed-all-2-sw',
 '1-EHVHV-mixed-all-0-sw',
 '1-EHVHV-mixed-all-0-no_sw',
 '1-EHVHV-mixed-all-1-sw',
 '1-EHVHV-mixed-all-1-no_sw',
 '1-EHVHV-mixed-all-2-sw',
 '1-EHVHV-mixed-all-2-no_sw',
 '1-EHVHV-mixed-1-0-sw',
 '1-EHVHV-mixed-1-0-no_sw',
 '1-EHVHV-mixed-1-1-sw',
 '1-EHVHV-mixed-1-1-no_sw',
 '1-EHVHV-mixed-1-2-sw',
 '1-EHVHV-mixed-1-2-no_sw',
 '1-EHVHV-mixed-2-0-sw',
 '1-EHVHV-mixed-2-0-no_sw',
 '1-EHVHV-mixed-2-1-sw',
 '1-EHVHV-mixed-2-1-no_sw',
 '1-EHVHV-mixed-2-2-sw',
 '1-EHVHV-mixed-2-2-no_sw',
 '1-EHV-mixed--0-sw',
 '1-EHV-mixed--0-no_sw',
 '1-EHV-mixed--1-sw',
 '1-EHV-mixed--1-no_sw',
 '1-EHV-mixed--2-sw',
 '1-EHV-mixed--2-no_sw',
 '1-HVMV-mixed-all-0-sw',
 '1-HVMV-mixed-all-0-no_sw',
 '1-HVMV-mixed-all-1-sw',
 '1-HVMV-mixed-all-1-no_sw',
 '1-HVMV-mixed-all-2-sw',
 '1-HVMV-mixed-all-2-no_sw',
 '1-HVM

In [ ]:
net = sb.get_simbench_net('1-HV-mixed--0-no_sw')#'1-HV-mixed--0-no_sw'

In [ ]:
in_channels = 4
hidden_channels = 256
out_channels = 4
activation = "elu"
scaler = StandardScaler()
num_layers = 5
dropout = 0.0
jk = "last"
lr = 0.00001
layer_type = "TransConv"
# torch_geometric.nn.norm.batch_norm.BatchNorm(hidden_channels)
#device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = GNN(in_channels, hidden_channels, num_layers, out_channels, dropout=dropout, norm=torch_geometric.nn.norm.batch_norm.BatchNorm(hidden_channels),jk=jk,layer_type=layer_type, activation=activation)#.to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=lr)

ordered_dict = torch.load(r"C:\Users\canba\OneDrive\Masaüstü\OPFGNN\code\Models\Supervised\supervisedmodel.pt")

model.load_state_dict(ordered_dict)

In [ ]:
'/'.join(os.path.dirname(os.path.abspath(__file__).split("\\"))) + r"/Models/SelfSupervised/base_model.pt"


In [ ]:
net.bus

In [ ]:
to_json('1-HV-mixed--0-no_sw')

In [ ]:
net = sb.get_simbench_net('1-HV-mixed--0-no_sw')#'1-HV-mixed--0-no_sw'
#Replace all ext_grids but the first one with generators and set the generators to slack= false
ext_grids = [i for i in range(1,len(net.ext_grid.name.values))]
pp.replace_ext_grid_by_gen(net,ext_grids=ext_grids, slack=False)

#TODO: reactive power limits for gens?


#TODO: reactive power limits for sgens?



#NETWORK CONSTRAINTS

#Maximize the branch limits

#max_i_ka = list(net.line.max_i_ka.values)

#for i in range(len(max_i_ka)):
# max_i_ka[i] = max(max_i_ka)



#Maximize line loading percents
max_loading_percent = list(net.line.max_loading_percent.values)
for i in range(len(max_loading_percent)):
    max_loading_percent[i] = 100.0
net.line.max_loading_percent = max_loading_percent

#Maximize trafo loading percent
max_loading_percent = list(net.trafo.max_loading_percent.values)
for i in range(len(max_loading_percent)):
    max_loading_percent[i] = 100.0
net.trafo.max_loading_percent = max_loading_percent

#Maximize trafo3w loading percent
max_loading_percent = list(net.trafo3w.max_loading_percent.values)
for i in range(len(max_loading_percent)):
    max_loading_percent[i] = 100.0
net.trafo3w.max_loading_percent = max_loading_percent

#Cost assignment

pp.create_pwl_costs(net, [i for i in range(len(net.gen.name.values))],et="gen", points=[[[0, 20, 1], [20, 30, 2]] for _ in range(len(net.gen.name.values))])
pp.create_pwl_costs(net, [i for i in range(len(net.sgen.name.values))],et="sgen", points=[[[0, 20, 0.25], [20, 30, 0.5]] for _ in range(len(net.sgen.name.values))])
pp.create_pwl_costs(net, [i for i in range(len(net.ext_grid.name.values))],et="ext_grid", points=[[[0, 20, 2], [20, 30, 5]] for _ in range(len(net.ext_grid.name.values))])

pp.runopp(net,verbose=True)
net.res_bus

In [ ]:
train_data, val_data, test_data = read_unsupervised_dataset('1-HV-mixed--0-no_sw')

In [ ]:
#train_data[0].has_isolated_nodes()
#train_data[0].has_self_loops()
#train_data[0].is_undirected()
x_dict = train_data[0].to_dict()
#to_json(train_dict)
ln = len(x_dict[("NB","-", "NB")]["edge_index"][0])
print(x_dict[("NB","-", "NB")]["edge_index"])#[:, :int(ln/2)]
x_dict

In [ ]:
net = sb.get_simbench_net('1-HV-mixed--0-no_sw')
idx_mapper, node_types_as_dict = extract_node_types_as_dict(net)
for key in node_types_as_dict:
    print(f"Bus Type: {key}")
    for i in range(len(node_types_as_dict[key])):
        #node_types_as_dict[key][i] = idx_mapper[node_types_as_dict[key][i]]
        print(str(node_types_as_dict[key][i]))
    print("-------------------------------")

In [ ]:
x_dict["PQ"]['x'][2]

In [ ]:
grids_available = []
for nw_name in all_simbench_codes:
        net = sb.get_simbench_net(nw_name)
        print("Trying Network named " + nw_name + "...")

        #dict_probs = pp.diagnostic(net,report_style='None')
        #for bus_num in dict_probs['multiple_voltage_controlling_elements_per_bus']['buses_with_gens_and_ext_grids']:
        #    net.gen = net.gen.drop(net.gen[net.gen.bus == bus_num].index)

        #OPERATIONAL CONSTRAINTS

        #Set upper and lower limits of active-reactive powers of loads
        min_p_mw_val, max_p_mw_val, min_q_mvar_val, max_q_mvar_val = [], [], [], []
        p_mw = list(net.load.p_mw.values)
        q_mvar = list(net.load.q_mvar.values)

        for i in range(len(p_mw)):
            min_p_mw_val.append(p_mw[i])
            max_p_mw_val.append(p_mw[i])
            min_q_mvar_val.append(q_mvar[i])
            max_q_mvar_val.append(q_mvar[i])

        net.load.min_p_mw = min_p_mw_val
        net.load.max_p_mw = max_p_mw_val
        net.load.min_q_mvar = min_q_mvar_val
        net.load.max_q_mvar = max_q_mvar_val

        #Replace all ext_grids but the first one with generators and set the generators to slack= false
        ext_grids = [i for i in range(1,len(net.ext_grid.name.values))]
        pp.replace_ext_grid_by_gen(net,ext_grids=ext_grids, slack=False)

        #TODO: reactive power limits for gens?


        #TODO: reactive power limits for sgens?



        #NETWORK CONSTRAINTS

        #Maximize the branch limits

        #max_i_ka = list(net.line.max_i_ka.values)

        #for i in range(len(max_i_ka)):
        # max_i_ka[i] = max(max_i_ka)



        #Maximize line loading percents
        max_loading_percent = list(net.line.max_loading_percent.values)
        for i in range(len(max_loading_percent)):
            max_loading_percent[i] = 100.0
        net.line.max_loading_percent = max_loading_percent

        #Maximize trafo loading percent
        max_loading_percent = list(net.trafo.max_loading_percent.values)
        for i in range(len(max_loading_percent)):
            max_loading_percent[i] = 100.0
        net.trafo.max_loading_percent = max_loading_percent

        #Maximize trafo3w loading percent
        max_loading_percent = list(net.trafo3w.max_loading_percent.values)
        for i in range(len(max_loading_percent)):
            max_loading_percent[i] = 100.0
        net.trafo3w.max_loading_percent = max_loading_percent

        #Cost assignment
        pp.create_pwl_costs(net, [i for i in range(len(net.gen.name.values))],et="gen", points=[[[0, 20, 1], [20, 30, 2]] for _ in range(len(net.gen.name.values))])
        pp.create_pwl_costs(net, [i for i in range(len(net.sgen.name.values))],et="sgen", points=[[[0, 20, 0.25], [20, 30, 0.5]] for _ in range(len(net.sgen.name.values))])
        pp.create_pwl_costs(net, [i for i in range(len(net.ext_grid.name.values))],et="ext_grid", points=[[[0, 20, 2], [20, 30, 5]] for _ in range(len(net.ext_grid.name.values))])

        try:
            pp.runpm_dc_opf(net) # Run DCOPP
        except pp.OPFNotConverged:
            text = "DC OPTIMAL POWERFLOW COMPUTATION DID NOT CONVERGE FOR NETWORK " + nw_name + ".SKIPPING THIS DATASET."
            print(text)
            continue
        print("GRID NAMED "+nw_name+" CONVERGES FOR DCOPF")
        grids_available.append(nw_name)


In [ ]:
for nw_name in grids_available:
    net = sb.get_simbench_net(nw_name)
    print(nw_name + ": Number of Buses = " + str(len(net.bus))) # dcopp on all grids directly

In [ ]:
grids_ready = []
for nw_name in all_simbench_codes[6:]:
    net = sb.get_simbench_net(nw_name)
    print("Trying Network named " + nw_name + "...")

    #OPERATIONAL CONSTRAINTS

    #Set upper and lower limits of active-reactive powers of loads
    min_p_mw_val, max_p_mw_val, min_q_mvar_val, max_q_mvar_val = [], [], [], []
    p_mw = list(net.load.p_mw.values)
    q_mvar = list(net.load.q_mvar.values)

    for i in range(len(p_mw)):
        min_p_mw_val.append(p_mw[i])
        max_p_mw_val.append(p_mw[i])
        min_q_mvar_val.append(q_mvar[i])
        max_q_mvar_val.append(q_mvar[i])

    net.load.min_p_mw = min_p_mw_val
    net.load.max_p_mw = max_p_mw_val
    net.load.min_q_mvar = min_q_mvar_val
    net.load.max_q_mvar = max_q_mvar_val

    #Replace all ext_grids but the first one with generators and set the generators to slack= false
    ext_grids = [i for i in range(1,len(net.ext_grid.name.values))]
    pp.replace_ext_grid_by_gen(net,ext_grids=ext_grids, slack=False)

    #TODO: reactive power limits for gens?


    #TODO: reactive power limits for sgens?



    #NETWORK CONSTRAINTS

    #Maximize the branch limits

    #max_i_ka = list(net.line.max_i_ka.values)

    #for i in range(len(max_i_ka)):
    # max_i_ka[i] = max(max_i_ka)



    #Maximize line loading percents
    max_loading_percent = list(net.line.max_loading_percent.values)
    for i in range(len(max_loading_percent)):
        max_loading_percent[i] = 100.0
    net.line.max_loading_percent = max_loading_percent

    #Maximize trafo loading percent
    max_loading_percent = list(net.trafo.max_loading_percent.values)
    for i in range(len(max_loading_percent)):
        max_loading_percent[i] = 100.0
    net.trafo.max_loading_percent = max_loading_percent

    #Maximize trafo3w loading percent
    max_loading_percent = list(net.trafo3w.max_loading_percent.values)
    for i in range(len(max_loading_percent)):
        max_loading_percent[i] = 100.0
    net.trafo3w.max_loading_percent = max_loading_percent

    #Cost assignment
    pp.create_pwl_costs(net, [i for i in range(len(net.gen.name.values))],et="gen", points=[[[0, 20, 1], [20, 30, 2]] for _ in range(len(net.gen.name.values))])
    pp.create_pwl_costs(net, [i for i in range(len(net.sgen.name.values))],et="sgen", points=[[[0, 20, 0.25], [20, 30, 0.5]] for _ in range(len(net.sgen.name.values))])
    pp.create_pwl_costs(net, [i for i in range(len(net.ext_grid.name.values))],et="ext_grid", points=[[[0, 20, 2], [20, 30, 5]] for _ in range(len(net.ext_grid.name.values))])

    #ac_converged = True

    #start_vec_name = ""
    #for init in ["pf", "flat", "results"]:
    #    try:
    #        pp.runopp(net, init=init)  # Calculate ACOPF with IPFOPT
    #    except pp.OPFNotConverged:
    #        if init == "results":
    #            text = "AC OPTIMAL POWERFLOW COMPUTATION DID NOT CONVERGE FOR NETWORK " + nw_name + ". SKIPPING THIS GRID."
    #            print(text)
    #            break
    #        continue
    #    start_vec_name = init
    #    ac_converged = True
    #    break
    #if ac_converged:
    #    print("GRID NAMED "+nw_name+" CONVERGES FOR ACOPF" + "WITH THE START VECTOR OPTION " + start_vec_name + ".")


    try:
        pp.runpm_ac_opf(net) # Run DCOPP
    except pp.OPFNotConverged:
        text = "AC OPTIMAL POWERFLOW COMPUTATION DID NOT CONVERGE FOR NETWORK " + nw_name + ".SKIPPING THIS DATASET."
        print(text)
        continue
    print("GRID NAMED "+nw_name+" CONVERGES FOR ACOPF" + "WITH THE START VECTOR OPTION " + ".")
    grids_ready.append(nw_name)


In [ ]:
for nw_name in grids_ready:
    net = sb.get_simbench_net(nw_name)
    print(nw_name + ": Number of Buses = " + str(len(net.bus))) #julia acopf on all grids directly

In [ ]:
for nw_name in grids_ready:
    net = sb.get_simbench_net(nw_name)
    print(nw_name + ": Number of Buses = " + str(len(net.bus))) #pp acopf on all grids directly

In [ ]:
for nw_name in grids_ready:
    net = sb.get_simbench_net(nw_name)
    print(nw_name + ": Number of Buses = " + str(len(net.bus))) #julia acopf

In [ ]:
for nw_name in grids_ready:
    net = sb.get_simbench_net(nw_name)
    print(nw_name + ": Number of Buses = " + str(len(net.bus))) #pp acopf

In [ ]:
dcopf_acopf_available_grid_names = ["1-HV-mixed--0-no_sw","1-HV-urban--0-no_sw", "1-MV-comm--0-no_sw", "1-MV-semiurb--0-no_sw"]

In [ ]:
datasets_df_as_list = []
for grid_name in dcopf_acopf_available_grid_names:
    print("Generating datasets for grid " + grid_name)
    while len(datasets_df_as_list) != 100:
        net = sb.get_simbench_net(grid_name)
        df = create_dataset_from_dcopf_and_acopf(net)
        if df is not None:
            datasets_df_as_list.append(df)
            print(str(len(datasets_df_as_list)) + " Dataset(s) Generated" + " for the grid " + grid_name)

    print("Saving the datasets for grid " + grid_name)
    path = os.path.dirname(os.path.abspath("gnn.ipynb")) + "\\data\\Supervised\\Training\\" + grid_name
    if not os.path.exists(path):
        os.makedirs(path)

    for i in range(0, len(datasets_df_as_list)):
        newpath = path + "\\Dataset-" + str(i) + ".csv"
        datasets_df_as_list[i].to_csv(newpath)

    datasets_df_as_list.clear()

In [ ]:
dcopf_available_grid_names = ["1-HVMV-mixed-all-0-no_sw", "1-HVMV-mixed-1.105-0-no_sw", "1-HVMV-mixed-2.102-0-no_sw","1-HVMV-mixed-4.101-0-no_sw", "1-HVMV-urban-all-0-no_sw", "1-HVMV-urban-2.203-0-no_sw", "1-HVMV-urban-3.201-0-no_sw", "1-HVMV-urban-4.201-0-no_sw", "1-HV-mixed--0-no_sw", "1-HV-urban--0-no_sw", "1-MVLV-rural-all-0-no_sw"]

In [ ]:
datasets_df_as_list = []
for grid_name in dcopf_available_grid_names:
    print("Generating datasets for grid " + grid_name)
    while len(datasets_df_as_list) != 100:
        net = sb.get_simbench_net(grid_name)
        df = create_dataset_from_dcopf(net)
        if df is not None:
            datasets_df_as_list.append(df)
            print(str(len(datasets_df_as_list)) + " Dataset(s) Generated" + " for the grid " + grid_name)

    print("Saving the datasets for grid " + grid_name)
    path = os.path.dirname(os.path.abspath("gnn.ipynb")) + "\\data\\Unsupervised\\Training\\" + grid_name
    if not os.path.exists(path):
        os.makedirs(path)

    for i in range(0, len(datasets_df_as_list)):
        newpath = path + "\\Dataset-" + str(i) + ".csv"
        datasets_df_as_list[i].to_csv(newpath)

    datasets_df_as_list.clear()

In this revised version of the compute_node_embeddings function, the node features are weighted by the reverse admittance values in the adjacency matrix before they are summed to compute the node embeddings. The resulting node embeddings will reflect the strength of the connections between the nodes.

To use the reverse admittance values as the edge weights, you would need to pass the Ybus matrix as the adjacency matrix when calling the compute_node_embeddings function. The Ybus matrix should be converted to a PyTorch tensor before passing it to the function.

use the reverse admittance values as edge weights, you can modify the computation of the node embeddings to weight the node features by the reverse admittance values.

# Define the node types
node_types = ['Slack Node', 'Generator Node', 'Load Node']

# Define the number of nodes of each type in the graph
num_nodes = {
    'Slack Node': 1,
    'Generator Node': 20,
    'Load Node': 99
}


In [ ]:
grid_names = [_ for _ in os.listdir(os.path.dirname(os.path.abspath("gnn.ipynb")) + "\\data\\Supervised\\")]


In [ ]:
graphdata_lst = read_multiple_supervised_datasets(grid_names)

In [5]:
in_channels = 4
hidden_channels = 256
out_channels = 4
activation = "elu"
scaler = StandardScaler()
num_layers = 5
dropout = 0.0
jk = "last"
lr = 0.00001
layer_type = "TransConv"
# torch_geometric.nn.norm.batch_norm.BatchNorm(hidden_channels)
#device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = GNN(4, 256, num_layers, 4, dropout=0.0, norm=torch_geometric.nn.norm.batch_norm.BatchNorm(hidden_channels),jk="last",layer_type="TransConv", activation="elu")#.to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=lr)

ordered_dict = torch.load(r"C:\Users\canba\OneDrive\Masaüstü\OPFGNN\code\Models\Supervised\supervisedmodel.pt")

model.load_state_dict(ordered_dict)

<All keys matched successfully>

In [ ]:
lr

In [ ]:
test_datasets_lst = []
for i, graphdata in enumerate(graphdata_lst):
    run = wandb.init(
    # set the wandb project where this run will be logged
    project="OPF-GNN-Supervised-Homo",

    # track hyperparameters and run metadata
    config={
    "learning_rate": lr,
    "architecture": "Homo-GNN",
    "num_layers": num_layers,
    "dataset": grid_names[i],
    "epochs": 1000,
    "activation": activation,
    "dropout": dropout,
    "in_channels": in_channels,
    "output_channels": out_channels,
    "hidden_channels": hidden_channels,
    "channel type": layer_type,
    "norm": "BatchNorm",
    "scaler": scaler

    }
    )
    num_epochs = wandb.run.config.epochs
    run.watch(model)
    grid_name = graphdata.grid_name
    run.config.dataset = grid_name
    train_data = graphdata.train_data
    run.config["number of busses"] = np.shape(train_data[0].x)[0]
    val_data = graphdata.val_data
    test_data = graphdata.test_data
    test_datasets_lst.append(test_data)
    training_loader = DataLoader(train_data, batch_size=1, shuffle=True)
    validation_loader = DataLoader(val_data, batch_size=1, shuffle=True)
    #test_loader = DataLoader(test_data, batch_size=1, shuffle=True)
    for _ in range(num_epochs):
        #train_one_epoch(i, optimizer, training_loader, model, nn.MSELoss(), edge_index, edge_weights)
        train_validate_one_epoch(_, grid_name, optimizer, training_loader, validation_loader, model, nn.MSELoss(), scaler)
    print("Training and Validation finished " + "for GraphData " + str(i) + ".")


In [ ]:
train_loader_lst = []
val_loader_lst = []
test_loader_lst = []
for i, graphdata in enumerate(graphdata_lst):

    # Divide training data into chunks, load into Dataloaders and append to the list of training loaders
    for train_data in divide_chunks(graphdata.train_data, 5):
        train_loader_lst.append(DataLoader(train_data, batch_size=1, shuffle=True))

    # Divide validation data into chunks, load into Dataloaders and append to the list of validation loaders
    for val_data in divide_chunks(graphdata.val_data, 5):
        val_loader_lst.append(DataLoader(val_data, batch_size=1, shuffle=True))

    # Append the test data to the list
    for test_data in divide_chunks(graphdata.test_data, 5):
        test_loader_lst.append(DataLoader(test_data, batch_size=1, shuffle=True))

print("Data Preparation finished.")

In [ ]:
optimizer = torch.optim.Adam(model.parameters(), lr=lr/100)

In [ ]:
run = wandb.init(
    # set the wandb project where this run will be logged
    project="OPF-GNN-Supervised-Homo",

    # track hyperparameters and run metadata
    config={
    "learning_rate": lr,
    "architecture": "Homo-GNN",
    "num_layers": num_layers,
    "epochs": 2000,
    "activation": activation,
    "dropout": dropout,
    "in_channels": in_channels,
    "output_channels": out_channels,
    "hidden_channels": hidden_channels,
    "channel type": layer_type,
    "norm": "BatchNorm",
    "scaler": scaler
    }
    )
"""
for _ in range(wandb.run.config.epochs):
    # Training
    random.shuffle(train_loader_lst)
    print("Training the model for epoch " + str(_))
    train_all_one_epoch(_, optimizer, train_loader_lst, model, nn.MSELoss())

    # Validation
    random.shuffle(val_loader_lst)
    print("Validating the model for epoch " + str(_))
    validate_all_one_epoch(_, val_loader_lst, model, nn.MSELoss())

    outputs = test_all_one_epoch(test_loader_lst, model, nn.MSELoss())
print("Training and Validation finished.")
"""


In [ ]:
outputs = test_all_one_epoch(test_loader_lst, model, nn.MSELoss())

In [ ]:
output,target = outputs[0]

In [ ]:
path = r"C:\Users\canba\OneDrive\Masaüstü\OPFGNN\code\Models\Supervised\supervisedmodel.pt" #os.path.dirname(os.path.abspath("gnn.ipynb")) + "\\Models\\Supervised\\" + "basemodel.pt" #r"C:\Users\canba\OneDrive\Masaüstü\OPFGNN\code\Models\Supervised"
torch.save(model.state_dict(), "supervisedmodel.pt")
print("done")



In [ ]:
'/'.join(os.path.dirname(os.path.abspath("gnn.ipynb")).split("\\"))+ r"/Models/SelfSupervised/base_model.pt"

In [ ]:
from pandapower.plotting.simple_plot import simple_plot
from pandapower.plotting.plotly.simple_plotly import simple_plotly
#simple_plot(net, plot_gens=True, plot_loads=True, plot_sgens=True, library="igraph")
simple_plotly(net, map_style="satellite")

In [ ]:
pp.runpm_ac_opf(net)
net.res_bus

In [ ]:
pp.runpm_ac_opf(net)
net.res_bus

In [ ]:
pp.runopp(net)
net.res_bus

In [ ]:
idx_mapper, node_types_as_dict = extract_node_types_as_dict(net)
node_types_as_dict

In [4]:
in_channels = 4
hidden_channels = 256
out_channels = 4
activation = "elu"
scaler = StandardScaler()
num_layers = 5
dropout = 0.0
jk = "last"
lr = 0.00001
layer_type = "TransConv"
# torch_geometric.nn.norm.batch_norm.BatchNorm(hidden_channels)
#device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = GNN(in_channels, hidden_channels, num_layers, out_channels, dropout=dropout, norm=torch_geometric.nn.norm.batch_norm.BatchNorm(hidden_channels),jk=jk,layer_type=layer_type, activation=activation)#.to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=lr)

ordered_dict = torch.load(r"C:\Users\canba\OneDrive\Masaüstü\OPFGNN\code\Models\Supervised\supervisedmodel.pt")

model.load_state_dict(ordered_dict)

<All keys matched successfully>

In [5]:
model.parameters

<bound method Module.parameters of GNN(4, 4, num_layers=5)>

In [ ]:
grid_name = '1-HV-mixed--0-no_sw'

In [ ]:
pp.runpm_ac_opf(net)


In [ ]:
grid_name = '1-HV-mixed--0-no_sw'
net = process_network(grid_name)


In [ ]:
pp.runpm_ac_opf(net)
net.res_bus

 HETEROGENEOUS GNN

In [6]:
index_mappers,net,data = generate_unsupervised_input('1-HV-mixed--0-no_sw')

In [3]:
data["SB"].x

tensor([[ 380.,    0.,    0.,    0.,  342.,  418.,  350., -350.,  350., -350.,
          350.]], dtype=torch.float64, grad_fn=<StackBackward0>)

In [ ]:
model = create_ACOPFGNN_model(data)

In [7]:
x_dict, constraint_dict, edge_idx_dict, edge_attr_dict, bus_idx_neighbors_dict, scaler, angle_params, res_bus_dict = extract_unsupervised_inputs(data, net, index_mappers)

no costs are given - overall generated power is minimized
no costs are given - overall generated power is minimized


In [8]:
x_dict

{'SB': tensor([[ 5.3624, -1.1287,  0.1318,  0.1443]], requires_grad=True),
 'PQ': tensor([[-0.2242, -0.9261,  0.1855,  0.1735],
         [-0.2242, -0.9261,  0.1876,  0.1740],
         [-0.2242, -0.9261,  0.5085,  0.3566],
         [-0.2242, -0.9261,  0.8787,  0.5507],
         [-0.2242, -0.9261,  0.1823,  0.1737],
         [-0.2242, -0.9261,  0.1851,  0.1765],
         [-0.2242, -0.9261,  0.7988,  0.5809],
         [-0.2242, -0.9261,  0.1863,  0.1734],
         [-0.2242, -0.9261,  0.1867,  0.1750],
         [-0.2242, -0.9261,  0.1859,  0.1759],
         [-0.2242, -0.9261,  0.5017,  0.3234],
         [-0.2242, -0.9261,  0.4428,  0.3478],
         [-0.2242, -0.9261,  0.1850,  0.1719],
         [-0.2242, -0.9261,  0.1877,  0.1745],
         [-0.2242, -0.9261,  0.1855,  0.1764]], requires_grad=True),
 'PV': tensor([[ 5.3624e+00, -9.2614e-01, -2.8817e+00, -4.3336e+00],
         [ 2.0518e+00, -9.2614e-01, -2.4512e+00, -3.6939e+00],
         [-2.2418e-01, -9.2614e-01,  5.7386e-02,  1.0016e-01

In [10]:
embedder_model(x_dict, constraint_dict, edge_idx_dict, edge_attr_dict)

defaultdict(torch.Tensor,
            {'PQ': tensor([[-0.9727,  0.4975,  0.1855,  0.1735],
                     [-0.9881, -0.3441,  0.1876,  0.1740],
                     [-0.9433, -0.1555,  0.5085,  0.3566],
                     [-0.8736, -0.6474,  0.8787,  0.5507],
                     [-0.9996,  2.1459,  0.1823,  0.1737],
                     [-0.7947, -0.3202,  0.1851,  0.1765],
                     [-1.0000,  6.4361,  0.7988,  0.5809],
                     [-0.8129, -0.4006,  0.1863,  0.1734],
                     [-0.9954,  4.7195,  0.1867,  0.1750],
                     [-1.0000,  3.5074,  0.1859,  0.1759],
                     [-0.9077, -0.3457,  0.5017,  0.3234],
                     [-0.9938,  0.8982,  0.4428,  0.3478],
                     [ 0.1649,  0.0599,  0.1850,  0.1719],
                     [-0.9517,  1.1979,  0.1877,  0.1745],
                     [-0.9716,  0.3346,  0.1855,  0.1764]], grad_fn=<CatBackward0>),
             'NB': tensor([[-0.5042, -0.9679,  0.1318,  0

In [6]:
np.sqrt(0.167**2 + 0.1685**2)

0.23723669614964715

In [7]:
for node_type in constraint_dict:
    print(custom_standard_inverse_transform(scaler, constraint_dict[node_type].detach().numpy()))

[[ 342.       418.       350.      -350.       350.      -350.
   350.00003]]
[[ 99.        121.          3.1069336   2.8781586   2.8781586   1.1700897
    1.1700897]
 [ 99.        121.          3.257019    3.0235138   3.0235138   1.2110443
    1.2110443]
 [ 99.        121.         37.16533     8.023071   34.423065    3.5772247
   14.011185 ]
 [ 99.        121.         51.020508   33.0876     46.6476     15.307114
   20.66623  ]
 [ 99.        121.          3.0006409   2.704483    2.704483    1.2998047
    1.2998047]
 [ 99.        121.          3.2549133   3.059372    3.059372    1.1112213
    1.1112213]
 [ 99.        121.         49.814194   31.622696   45.182693   15.616501
   20.975632 ]
 [ 99.        121.          3.423462    3.208374    3.208374    1.1942902
    1.1942902]
 [ 99.        121.          3.0209045   2.8138428   2.8138428   1.0991058
    1.0991058]
 [ 99.        121.          3.4769897   3.2351685   3.2351685   1.2740631
    1.2740631]
 [ 99.        121.         31.8847

In [ ]:
ACOPFOutput(x_dict, scalers_dict, None, index_mappers).output

In [ ]:
grid_names = sb.collect_all_simbench_codes()[25:]
acopf_ok_grid_names = []

for grid_name in grid_names:
    print(f"Trying grid {grid_name}")
    net = process_network(grid_name)

    # try:
    #     acopf_ok_grid_names.append(grid_name)
    #     pp.runpm_ac_opf(net)
    #     #print(net.res_bus)
    # except:
    #     print(f"Julia didnt converge for {grid_name}")
    #     acopf_ok_grid_names.remove(grid_name)
    try:
        acopf_ok_grid_names.append(grid_name)
        pp.runopp(net)
        #print(net.res_bus)
    except:
        print(f"OPP didnt converge for {grid_name}")
        acopf_ok_grid_names.remove(grid_name)
acopf_ok_grid_names
#save_multiple_unsupervised_inputs(grid_names, 100)

In [15]:
acopf_ok_grid_names = ['1-HV-mixed--1-sw','1-MV-semiurb--0-no_sw','1-MV-comm--0-sw','1-MV-comm--0-no_sw']
net = process_network(acopf_ok_grid_names[1])
pp.runopp(net)
net.res_bus

no costs are given - overall generated power is minimized


,vm_pu,va_degree,p_mw,q_mvar,lam_p,lam_q
0,1.025000,0.000000,-8.057173,-11.835759,1.000000,2.259515e-21
2,1.001425,209.119600,-1.176327,0.135443,1.000714,1.887024e-03
4,0.999942,209.185313,0.397800,0.183689,1.002495,4.950700e-03
5,0.999154,209.227646,0.293150,0.148678,1.003357,6.811706e-03
6,0.998564,209.265084,0.239906,0.171257,1.003935,8.382988e-03
...,...,...,...,...,...,...
112,0.987459,209.245271,0.237492,0.096853,1.020719,1.774877e-02
113,0.987333,209.251486,0.036984,0.085200,1.020904,1.802879e-02
114,0.987261,209.254787,-0.074431,0.034476,1.021013,1.818089e-02
115,0.987149,209.258063,0.118649,0.137547,1.021198,1.835478e-02


In [ ]:
acopf_grid_names = ['1-HV-mixed--0-no_sw','1-MV-semiurb--0-no_sw', '1-MV-comm--0-no_sw']
save_multiple_unsupervised_inputs(acopf_grid_names, num_samples=100)

In [4]:
inputs = load_multiple_unsupervised_inputs()
for i,input in enumerate(inputs):
    if input.res_bus is None:
        print(f"None at {i}")

In [ ]:
save_unsupervised_inputs('1-HV-mixed--0-no_sw', 200)

In [ ]:
inputs = load_unsupervised_inputs('1-HV-mixed--0-no_sw')
inputs[0].res_bus

In [ ]:
inputs

In [ ]:
inputs[57].res_bus

Display Plot of Customized Sigmoid Function

In [ ]:
import matplotlib.pyplot as p
import torch

def custom_tanh(x: torch.Tensor, lower_bound: float, upper_bound: float) -> torch.Tensor:
    width = upper_bound - lower_bound
    return 0.5 * width * torch.tanh(x) + 0.5 * (upper_bound + lower_bound)


lst = []
min_val = 0
max_val = 0.1
range_ = []
i = -2
while i < 2:
    range_.append(i)
    i += 0.01

for i in range_:
    val = custom_tanh(torch.tensor(i), min_val, max_val)
    #val = torch.tanh(torch.tensor(i))
    lst.append(val)

# Now, plot the values in lst
p.plot(range_, lst)
p.xlabel('Input')
p.ylabel('Value')
p.title('Plot of Enforcing Activation Function')
p.grid(True)
p.show()
print(range_)

Create Heterogeneous Self Supervised Model and Process Inputs

In [2]:
# Define the Parameters
grid_name = '1-HV-mixed--0-no_sw'


In [3]:
# Load the Model
embedder_model = load_ACOPFGNN_model(grid_name, "embedder_model.pt", hidden_channels=4, num_layers=4)

no costs are given - overall generated power is minimized
no costs are given - overall generated power is minimized


defaultdict(<class 'list'>, {'PQ': tensor([[-0.3404,  0.4297,  0.2627, -0.3390],
        [ 5.2914, -0.7396,  1.2969,  4.7733],
        [-0.4595,  0.5880,  0.2941, -0.4409],
        [-0.5007,  0.6194,  0.3256, -0.4599],
        [ 1.6187, -0.5973,  0.4409,  1.5799],
        [-0.4982,  0.6073,  0.2925, -0.4751],
        [ 1.0997, -0.6364,  0.1321,  0.9262],
        [ 0.7430, -0.4017,  0.3454,  0.8069],
        [-0.4755,  0.6062,  0.3273, -0.4384],
        [ 0.1863, -0.1025,  0.1672,  0.0295],
        [ 4.7190, -0.8107,  0.9037,  4.1483],
        [-0.4805,  0.6122,  0.3222, -0.4476],
        [ 0.1433, -0.0297,  0.3895,  0.3056],
        [ 0.9695,  0.0323,  0.7686,  1.0610],
        [-0.4516,  0.5641,  0.2389, -0.4691],
        [-0.4463,  0.5778,  0.2557, -0.4572]], grad_fn=<EluBackward0>), 'PV': tensor([[-0.3094, -0.5684,  0.2777,  0.2449],
        [-0.0909, -0.0795,  0.1359, -0.1535],
        [ 0.2182,  0.5203,  0.3709,  0.0032],
        [ 0.4915,  0.4823,  0.2856,  0.0581],
        [ 0.1

RuntimeError: Error(s) in loading state_dict for ACOPFGNN:
	Missing key(s) in state_dict: "lin_2.weight", "lin_2.bias", "fcs.0.SB.weight", "fcs.0.SB.bias", "fcs.0.PQ.weight", "fcs.0.PQ.bias", "fcs.0.PV.weight", "fcs.0.PV.bias", "fcs.0.NB.weight", "fcs.0.NB.bias", "fcs.1.SB.weight", "fcs.1.SB.bias", "fcs.1.PQ.weight", "fcs.1.PQ.bias", "fcs.1.PV.weight", "fcs.1.PV.bias", "fcs.1.NB.weight", "fcs.1.NB.bias", "fcs.2.SB.weight", "fcs.2.SB.bias", "fcs.2.PQ.weight", "fcs.2.PQ.bias", "fcs.2.PV.weight", "fcs.2.PV.bias", "fcs.2.NB.weight", "fcs.2.NB.bias", "fcs.3.SB.weight", "fcs.3.SB.bias", "fcs.3.PQ.weight", "fcs.3.PQ.bias", "fcs.3.PV.weight", "fcs.3.PV.bias", "fcs.3.NB.weight", "fcs.3.NB.bias". 

In [3]:
# Create ACOPFGNN model and Optimizer
index_mappers,net,data = generate_unsupervised_input(grid_name)
model = create_ACOPFGNN_model(data, net, index_mappers, hidden_channels=4, num_layers=1)

no costs are given - overall generated power is minimized
no costs are given - overall generated power is minimized


defaultdict(<class 'list'>, {'SB': tensor([[ 0.1041,  0.1142, -0.4791,  0.7446]], grad_fn=<EluBackward0>), 'PQ': tensor([[ 0.2601,  0.0198, -0.0580,  0.4459],
        [ 0.2241,  0.0918,  0.0923,  0.5312],
        [ 0.3650, -0.1676, -0.0951,  0.2352],
        [ 0.4566, -0.2931, -0.1155,  0.0503],
        [ 0.0474,  0.4948,  0.1282,  0.8651],
        [ 0.2946, -0.0845, -0.0996,  0.3877],
        [ 0.3538, -0.2359, -0.2082,  0.2790],
        [ 0.2371,  0.0829, -0.0202,  0.4880],
        [ 0.3421, -0.1565, -0.1316,  0.2869],
        [ 0.2901, -0.3365, -0.3776,  0.4726],
        [ 0.0320,  0.6127,  0.1320,  0.8688],
        [ 0.3712, -0.1951, -0.1204,  0.2278],
        [ 0.2325,  0.1152,  0.0070,  0.4927],
        [ 0.4929, -0.3178, -0.3468, -0.0792],
        [ 0.3148, -0.1695, -0.1970,  0.3557],
        [ 0.3062, -0.0926, -0.1085,  0.3589]], grad_fn=<EluBackward0>), 'NB': tensor([[-0.1595,  0.3800, -0.1613,  1.4756],
        [-0.0402,  0.2727, -0.4049,  1.0814],
        [-0.1306,  0.3673, 

In [4]:
num_parameters = sum(p.numel() for p in model.parameters() if p.requires_grad)
learning_rate = 2.5e-4
optimizer = torch.optim.Adam(model.parameters())

In [5]:
num_parameters

2321

In [5]:
WANDB_NOTEBOOK_NAME = "msi"
# Initialize Weights & Biases
run = wandb.init(
        # set the wandb project where this run will be logged
        project="ACOPF-GNN-SelfSupervised-Hetero",

        # track hyperparameters and run metadata
        config={
            #"learning_rate": optimizer.,
            "architecture": "Hetero-SelfSupervised-GNN",
            "num_parameters": sum(p.numel() for p in model.parameters() if p.requires_grad),
            "num_heads": model.heads,
            "num_layers": model.num_layers,
            "grid_name": grid_name,
            "activation": model.act_fn,
            "dropout": model.dropout,
            "in_channels": model.in_channels,
            "output_channels": model.out_channels,
            "hidden_channels": model.hidden_channels,
            "channel type": "TransformerConv",
            "scaler": "MinMax",
            "model": "base_model_wired"
        }
    )


wandb: Currently logged in as: canbay96 (cbml). Use `wandb login --relogin` to force relogin


In [6]:
# Load Inputs
inputs = load_unsupervised_inputs(grid_name)
#inputs = load_multiple_unsupervised_inputs()
n = len(inputs)
train_inputs = inputs[:int(n*0.8)]
val_inputs = inputs[int(n*0.8): int(n*0.95)]
test_inputs = inputs[int(n*0.95):]



In [7]:
# Train and Validate Model
start_epoch = 1
num_epochs = 100
loss_weights = (0.0, 1.0, 0.0)
model_name = "embedder_model_specialized.pt"
train_input,train_output,val_input, val_output = train_validate_ACOPF(model, model_name, optimizer, train_inputs, val_inputs,loss_weights, start_epoch, num_epochs)

---------------   CURRENT LEARNING RATE: 0.001   ---------------
Epoch: 0
###########################################
   TRAINING
     Training Step: 0 Training Loss: 0.0 
     Training Step: 1 Training Loss: 0.0 
     Training Step: 2 Training Loss: 0.0 


KeyboardInterrupt: 

In [23]:
for i in range(61, 120, 20) :
    start_epoch = i
    num_epochs = 20 + i
    loss_weights = (1.0, 1.0, 1.0)
    model_name = "base_model_unsupervised_physics_constrained.pt"
    train_input,train_output,val_input, val_output = train_validate_ACOPF(model, model_name, optimizer, train_inputs, val_inputs,loss_weights, start_epoch, num_epochs)

---------------   CURRENT LEARNING RATE: 4e-05   ---------------
Epoch: 60
###########################################
   TRAINING
     Training Step: 0 Training Loss: 6.849725723266602 
     Training Step: 1 Training Loss: 7.994863033294678 
     Training Step: 2 Training Loss: 7.418996334075928 
     Training Step: 3 Training Loss: 4.887266635894775 
     Training Step: 4 Training Loss: 5.012649059295654 
     Training Step: 5 Training Loss: 3.0778064727783203 
     Training Step: 6 Training Loss: 11.675814628601074 
     Training Step: 7 Training Loss: 10.785894393920898 
     Training Step: 8 Training Loss: 8.542866706848145 
     Training Step: 9 Training Loss: 9.035882949829102 
     Training Step: 10 Training Loss: 9.545356750488281 
     Training Step: 11 Training Loss: 8.43463134765625 
     Training Step: 12 Training Loss: 12.55498218536377 
     Training Step: 13 Training Loss: 5.320827484130859 
     Training Step: 14 Training Loss: 6.62136173248291 
     Training Step: 15 

In [8]:
print(sum(train_output.output[:, 2]), sum(train_output.output[:, 3]))
train_output.output

-593.8486800193787 -243.53982478380203


array([[ 9.08173089e-01, -1.83501959e+00, -5.90927734e+01,
        -4.32426147e+01],
       [ 9.80060136e-01,  0.00000000e+00,  9.09702148e+01,
        -3.20258598e+01],
       [ 1.03807630e+00, -1.48351908e+00, -5.60315399e+01,
        -4.20662727e+01],
       [ 1.03758441e+00,  1.54328156e+00, -8.17986012e-01,
         7.37901390e-01],
       [ 1.02849822e+00,  1.60070038e+00,  2.79828215e+00,
         8.60434711e-01],
       [ 1.02379331e+00,  1.61828518e+00,  2.95484495e+00,
         1.07658315e+00],
       [ 1.05319075e+00,  1.20048451e+00, -6.75554800e+00,
        -1.33368182e+00],
       [ 1.04970037e+00,  9.24345732e-01, -1.38833284e+01,
        -2.18041611e+00],
       [ 1.05298337e+00,  1.13721991e+00, -8.29765701e+00,
        -1.68316507e+00],
       [ 1.03909489e+00,  2.39133453e+00,  2.30472603e+01,
         6.45330811e+00],
       [ 1.06325094e+00, -8.12291145e-01, -5.12167511e+01,
        -1.27561131e+01],
       [ 1.06275891e+00,  5.54805279e-01, -2.24758530e+01,
      

In [9]:
print(sum(val_output.output[:, 2]), sum(val_output.output[:, 3]))
val_output.output

-511.95279839634895 -215.83535793423653


array([[ 9.07937301e-01, -1.87421203e+00, -5.86987801e+01,
        -4.31370201e+01],
       [ 9.78537469e-01,  0.00000000e+00,  9.15387573e+01,
        -3.18711929e+01],
       [ 1.03759433e+00, -1.51700187e+00, -5.56474800e+01,
        -4.19558372e+01],
       [ 1.03786753e+00,  1.56929421e+00, -2.91552931e-01,
         5.73150635e-01],
       [ 1.02684687e+00,  1.61438823e+00,  2.79138613e+00,
         9.32665348e-01],
       [ 1.02212753e+00,  1.63119197e+00,  2.91438818e+00,
         1.13415098e+00],
       [ 1.05419103e+00,  1.20319247e+00, -6.55964136e+00,
        -1.39674544e+00],
       [ 1.05039056e+00,  9.21392202e-01, -1.37815561e+01,
        -2.24824166e+00],
       [ 1.05342338e+00,  1.15047312e+00, -7.85218763e+00,
        -1.60835695e+00],
       [ 1.03538874e+00,  2.41840339e+00,  2.30585728e+01,
         6.94025660e+00],
       [ 1.06390915e+00, -8.37588310e-01, -5.08798523e+01,
        -1.26762056e+01],
       [ 1.06576177e+00,  5.09796381e-01, -2.31083965e+01,
      

In [12]:
def save_ACOPFGNN_model(model: ACOPFGNN):
    path = r"./Models/SelfSupervised/embedder_model.pt"

    torch.save(model.state_dict(), path)
save_ACOPFGNN_model(embedder_model)

In [ ]:
train_output.output

CHAINED ACOPF MODELS

-

-

-

-

In [8]:
def load_ACOPFGNN_model(grid_name, model_name, hidden_channels, num_layers):
    path = r"./Models/SelfSupervised/" + model_name
    index_mappers, net, data = generate_unsupervised_input(grid_name)
    model = create_ACOPFEmbedder_model(data, net, index_mappers, hidden_channels=hidden_channels, num_layers=num_layers)

    # Load the saved model parameter
    ordered_dict = torch.load(path)
    model.load_state_dict(ordered_dict)

    return model
embedder_model = load_ACOPFGNN_model(grid_name, "embedder_model.pt",8,1)

no costs are given - overall generated power is minimized
no costs are given - overall generated power is minimized


defaultdict(<class 'list'>, {'PQ': tensor([[ 0.0357,  0.2132],
        [ 0.3546, -0.4051],
        [ 0.3367,  0.7079],
        [ 0.6779,  1.2298],
        [ 0.2732, -0.7593],
        [ 0.2899,  0.8922],
        [-0.0812, -0.6637],
        [ 0.0920, -0.6473],
        [ 0.3620,  0.6066],
        [-0.6855, -0.7911],
        [ 0.0236, -0.9724],
        [ 0.3322,  0.8732],
        [ 0.2427, -0.2146],
        [ 0.1472, -0.0646],
        [-0.0681,  0.3301],
        [ 0.0695,  0.5528]], grad_fn=<EluBackward0>), 'NB': tensor([[ 1.0562,  1.2268],
        [ 0.6800,  0.3162],
        [ 0.5115,  0.8864],
        [ 2.7919, -0.3620],
        [ 0.6721, -0.0118]], grad_fn=<EluBackward0>), 'SB': tensor([[-0.1552, -0.5344]], grad_fn=<EluBackward0>), 'PV': tensor([[ 3.4214,  1.1388],
        [ 2.3178,  0.5479],
        [ 0.3299, -0.1493],
        [-0.0295, -0.3002],
        [-0.3908, -0.2448],
        [ 1.1586,  4.4030],
        [-0.2975, -0.2438],
        [-0.3383,  1.8400],
        [-0.6998, -0.0276],
 

In [3]:
# Create Minimizer, Enforcer and Embedder Models
# Define the Parameters
grid_name = '1-HV-mixed--0-no_sw'
index_mappers,net,data = generate_unsupervised_input(grid_name)
#minimizer_model = create_ACOPFGNN_model(data, net, index_mappers, hidden_channels=4, num_layers=1)
#embedder_model = create_ACOPFGNN_model(data, net, index_mappers, hidden_channels=4, num_layers=1)

#minimizer_model = create_ACOPFGNN_model(data, net, index_mappers, hidden_channels=4, num_layers=4)

embedder_model = create_ACOPFEmbedder_model(data, net, index_mappers, hidden_channels=8, num_layers=1)

#enforcer_model = create_ACOPFEnforcer_model(data, net, index_mappers, hidden_channels=4, num_layers=4)

no costs are given - overall generated power is minimized
no costs are given - overall generated power is minimized


defaultdict(<class 'list'>, {'PQ': tensor([[ 0.7643, -0.2924],
        [ 1.3948, -0.3719],
        [ 0.4723, -0.4887],
        [ 0.1594, -0.6713],
        [ 2.5409,  0.4687],
        [ 0.3421, -0.3662],
        [ 1.0452,  1.2068],
        [ 0.6408, -0.3298],
        [-0.0715,  1.8799],
        [ 5.0214,  1.0806],
        [ 0.2718, -0.5127],
        [ 1.3767, -0.3152],
        [ 1.6039,  0.1640],
        [ 0.3523, -0.2573],
        [ 0.7157, -0.2564]], grad_fn=<EluBackward0>), 'NB': tensor([[-0.4999, -0.8842],
        [ 0.3462, -0.4735],
        [-0.3032, -0.8039],
        [-0.2253,  1.3591],
        [ 0.3844, -0.4709]], grad_fn=<EluBackward0>), 'SB': tensor([[0.9981, 0.0247]], grad_fn=<EluBackward0>), 'PV': tensor([[-0.3481, -0.5532],
        [-0.2016, -0.3629],
        [-0.1346, -0.4042],
        [ 0.2581,  0.2171],
        [ 0.1846,  0.1537],
        [-0.7190, -0.9295],
        [ 0.1374,  0.1848],
        [-0.3030, -0.5192],
        [ 0.1089, -0.0378],
        [ 0.1797,  0.1571],
   

In [9]:
# Initialize the Optimizer
from itertools import chain
#optimizer = torch.optim.Adam(chain(minimizer_model.parameters(), enforcer_model.parameters(), embedder_model.parameters()))
ACOPF_optimizer = torch.optim.Adam(embedder_model.parameters(), lr=1e-3)


In [54]:
ACOPF_optimizer.param_groups[0]['lr']*=0.5

In [5]:
WANDB_NOTEBOOK_NAME = "msi"
# Initialize Weights & Biases
run = wandb.init(
        # set the wandb project where this run will be logged
        project="ACOPF-GNN-SelfSupervised-Hetero",

        # track hyperparameters and run metadata
        config={
            #"learning_rate": optimizer.,
            "architecture": "Hetero-SelfSupervised-GNN",
            "grid_name": grid_name,
            "Training Type": "Chained Models"

        }
    )


wandb: Currently logged in as: canbay96 (cbml). Use `wandb login --relogin` to force relogin


In [10]:
# Load Inputs
inputs = load_unsupervised_inputs(grid_name)
#inputs = load_multiple_unsupervised_inputs()
n = len(inputs)
train_inputs = inputs[:int(n*0.8)]
val_inputs = inputs[int(n*0.8): int(n*0.95)]
test_inputs = inputs[int(n*0.95):]

In [11]:
# Train and Validate Model
start_epoch = 1
num_epochs = 30
loss_weights = (0.0, 1.0, 0.0)
train_input,train_output,val_input, val_output = train_validate_ACOPF_chained(None,None, embedder_model,ACOPF_optimizer, train_inputs, val_inputs,loss_weights, start_epoch, num_epochs)

Epoch: 0
###########################################
   TRAINING
P: Output= 0.3622663861472244 --- Target= -8.159233733096016e-08
Q: Output= 2.1193104213935827 --- Target= -2.2176131064810534e-07
     Training Step: 0 Training Loss: 0.21346211433410645 
P: Output= 0.8204967933171465 --- Target= 1.335692196846594e-07
Q: Output= 2.2066302509560902 --- Target= 6.76467504234779e-08
     Training Step: 1 Training Loss: 0.235146164894104 
P: Output= -0.07886565718975191 --- Target= 1.9959909547395682e-07
Q: Output= 0.49240111723549873 --- Target= -2.0505563647077452e-07
     Training Step: 2 Training Loss: 0.22146496176719666 
P: Output= 0.5511021996207015 --- Target= 7.188702522142876e-08
Q: Output= 1.6426029632760901 --- Target= -7.979786786904697e-08
     Training Step: 3 Training Loss: 0.23362955451011658 
P: Output= -0.29034264784450414 --- Target= 2.191291796904693e-07
Q: Output= 0.48771047467349415 --- Target= -5.284382709191959e-08
     Training Step: 4 Training Loss: 0.2202627360820

KeyboardInterrupt: 

In [ ]:
# Train and Validate Model
start_epoch = 6
num_epochs = 30
loss_weights = (1.0, 1.0, 1.0)
train_input,train_output,val_input, val_output = train_validate_ACOPF_chained(minimizer_model,enforcer_model, embedder_model,ACOPF_optimizer, train_inputs, val_inputs,loss_weights, start_epoch, num_epochs)

In [ ]:
print(sum(train_output.output[:, 2]), sum(train_output.output[:, 3]))
train_output.output

In [ ]:
print(sum(val_output.output[:, 2]), sum(val_output.output[:, 3]))
val_output.output